In [1]:
import cv2
import os
import sys
from datetime import datetime
from ultralytics import YOLO
import matplotlib.pyplot as plt

cwd = os.getcwd()
delimiter = "\\" if "\\" in cwd else "/"
repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")]) + delimiter
sys.path.append(repoPath + "dataImport/")
import vidUtils
from vidUtils import getFrame, getCap
import rwWorkingTSDf
from rwWorkingTSDf import readWorkingTSDF, writeWorkingTSDf, getWorkingTSDfPath

In [2]:
model = YOLO("yolo11x-pose.pt")

In [3]:
deviceDescriptor = ["abhik", "bedroompi", "abhik", "piCam-raspberryPi5-Camv3120", "mp4", "piVidCap"]
outputDescriptor = ["abhik", "bedroompi", "ultralytics", "piCam-raspberryPi5-Camv3120", "17pose", "yolo11x-pose"]

In [4]:
vidDf = readWorkingTSDF(*deviceDescriptor)
import json

newDfRows = []
for i in range(100):
    row = vidDf.iloc[i]
    ts = row.name
    videoIndex = row['videoIndex']
    videoStartTime = row['videoStartTime']
    videoEndTime = row['videoEndTime']
    frame = getFrame(deviceDescriptor, videoStartTime, videoEndTime, videoIndex)
    result = model(frame, verbose=False)
    toSave = json.dumps({'xyn': result[0].keypoints.xyn.tolist(), 'conf': result[0].keypoints.conf.tolist()})
    newDfRows.append([ts, toSave])


read in 30466 rows from 1 files, retruning 30466 rows


In [5]:
import pandas as pd
newDf = pd.DataFrame(data=newDfRows, columns=['sampleDT', 'pose'])
newDf = newDf.set_index('sampleDT')
newDf

,pose
sampleDT,
2025-03-13 23:03:00.000596+00:00,"{""xyn"": [[[0.548828661441803, 0.53396099805831..."
2025-03-13 23:03:00.100148+00:00,"{""xyn"": [[[0.5484716296195984, 0.5321729779243..."
2025-03-13 23:03:00.201352+00:00,"{""xyn"": [[[0.5490943193435669, 0.5313501954078..."
2025-03-13 23:03:00.303282+00:00,"{""xyn"": [[[0.5493457913398743, 0.5299937725067..."
2025-03-13 23:03:00.402564+00:00,"{""xyn"": [[[0.5500948429107666, 0.5293601155281..."
...,...
2025-03-13 23:03:09.500148+00:00,"{""xyn"": [[[0.5534456372261047, 0.5194787979125..."
2025-03-13 23:03:09.600136+00:00,"{""xyn"": [[[0.5525461435317993, 0.5217380523681..."
2025-03-13 23:03:09.700147+00:00,"{""xyn"": [[[0.5516005158424377, 0.5251188278198..."


In [ ]:
writeWorkingTSDf(outputDescriptor, newDf)

saved to a file named 2025-03-13T230300,000596+0000_2025-03-13T230309,900155+0000_821a0a3a_.parquet.gzip


In [ ]:
import duckdb




query = """
SELECT * FROM 'frames.parquet.gzip'
WHERE timestamp BETWEEN 100 AND 200
"""
result = duckdb.query(query).df()
print(result)